In [4]:
import pandas as pd

In [8]:
!pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from bs4 import BeautifulSoup as bs4

In [5]:
req= requests.get("https://tenders.ppa.gov.gh/tenders")
req.raise_for_status()
soup = bs4(req.text, 'html.parser')
print(soup.prettify()[:1000])  # Print the first 1000 characters of the HTML for inspection

<!DOCTYPE html>
<!--[if IE 8 ]><html class="ie" xmlns="http://www.w3.org/1999/xhtml" xml:lang="en-US" lang="en-US"> <![endif]-->
<!--[if (gte IE 9)|!(IE)]><!-->
<html lang="en-US" xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml">
 <!--<![endif]-->
 <head>
  <!-- Basic Page Needs -->
  <meta charset="utf-8"/>
  <!--[if IE]><meta http-equiv='X-UA-Compatible' content='IE=edge,chrome=1'><![endif]-->
  <title>
   PPA Ghana
  </title>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <link href="https://tenders.ppa.gov.gh/css/bootstrap4.min.css" rel="stylesheet" type="text/css"/>
  <link href="https://tenders.ppa.gov.gh/stylesheets/style.css" rel="stylesheet" type="text/css"/>
  <link href="https://tenders.ppa.gov.gh/css/helpers.css" rel="stylesheet" type="text/css">
   <link href="https://tenders.ppa.gov.gh/css/front.css" rel="stylesheet" type="text/css"/>
   <link href="images/icon/favicon.png" rel="shortcut icon"/>
   <!--[if lt IE 9]>
       